In [26]:
import os
import time
from datetime import datetime, timedelta
import tempfile
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [27]:
def parse_datetime(date):
    formats = ['%m/%d/%Y %I:%M:%S %p', '%m-%d-%Y %H:%M']
    # Format 1: '5/13/2025 12:00:00 AM'
    # Format 2: '05-12-2025 23:55'
    date = str(date).strip()
    for fmt in formats:
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            continue
    return pd.NaT


def clean_data(df, area='PJM RTO'):
    """
    Filters and standardizes actual load data for a given area.

    Parameters:
        df (pd.DataFrame): Raw actual data.
        area (str): Zone/region to filter. Default is 'PJM RTO'.

    Returns:
        pd.DataFrame: Cleaned actual data with standardized column names.
    """
    df = df[df['area'] == area]
    df = df[['datetime_beginning_utc', 'instantaneous_load']]
    df = df.rename(columns={
        'datetime_beginning_utc': 'timestamp',
        'instantaneous_load': 'load'
    })
    df['timestamp'] = df['timestamp'].apply(parse_datetime)

    failed = df['timestamp'].isna().sum()
    if failed:
        print(f"Warning: {failed} timestamps failed to parse and will be set as NaT")

    df = df.sort_values(by=['timestamp']).reset_index(drop=True)
    return df


In [28]:
class PJMDataFetcher:
    def __init__(self, download_dir: str = None, headless: bool = True):
        self.download_dir = download_dir or tempfile.mkdtemp()
        self.headless = headless
        self.download_wait = 30
        self.driver = self._setup_driver()

        os.makedirs(self.download_dir, exist_ok=True)

    def _setup_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument('--headless=new')
        prefs = {'download.default_directory': self.download_dir}
        options.add_experimental_option('prefs', prefs)
        return webdriver.Chrome(options=options)


    def _wait_for_file_download(self, before: set, timeout: int = 60) -> str:
        start = time.time()
        while True:
            files = set(os.listdir(self.download_dir))
            new_files = files - before
            finished = [f for f in new_files if not f.endswith('.crdownload')]
            if finished:
                return os.path.join(self.download_dir, finished[0])
            if time.time() - start > timeout:
                raise TimeoutError("Download timed out.")
            time.sleep(1)


    def _download_via_xpath(self, xpath: str, url: str) -> pd.DataFrame:
        self.driver.get(url)
        time.sleep(5)  # Wait for page load
        before = set(os.listdir(self.download_dir))
        self.driver.find_element(By.XPATH, xpath).click()
        file_path = self._wait_for_file_download(before, timeout=self.download_wait)
        df = pd.read_csv(file_path)
        os.remove(file_path)
        return df


    def _force_fill_time(self, input_element, value: str):
        """Sets input value via JS and triggers Angular-compatible events."""
        self.driver.execute_script("""
            const el = arguments[0];
            const val = arguments[1];
            el.value = val;
            el.dispatchEvent(new Event('input', { bubbles: true }));
            el.dispatchEvent(new Event('change', { bubbles: true }));
            el.dispatchEvent(new Event('blur', { bubbles: true }));
            el.dispatchEvent(new Event('keyup', { bubbles: true }));
        """, input_element, value)


    def fetch_recent_actual(self) -> pd.DataFrame:
        """Fetches recent actual instantaneous load data."""

        actual = '/html/body/app-root/main/dm-feed-main/div/div/div/section/dm-feed-list/div[2]/main/dm-frequently-accessed/div/div[2]/div[3]/div[2]/ul/li/span[2]/span[3]/dm-feed-download-link/a'
        url = 'https://dataminer2.pjm.com/list'

        return self._download_via_xpath(actual, url)


    def fetch_recent_forecast(self) -> pd.DataFrame:
        """Clicks feed download for latest forecast"""

        forecast = '/html/body/app-root/main/dm-feed-main/div/div/div/section/dm-feed-list/div[2]/main/dm-frequently-accessed/div/div[2]/div[4]/div[2]/ul/li[3]/span[2]/span[3]/dm-feed-download-link/a'
        url = 'https://dataminer2.pjm.com/list'

        return self._download_via_xpath(forecast, url)

    def fetch_historical_actual(self) -> pd.DataFrame:
        """
        Fills 6-hour historical time window, submits, waits for paginator,
        exports CSV, and returns as DataFrame.
        """

        url = 'https://dataminer2.pjm.com/feed/inst_load'
        self.driver.get(url)

        # Wait for inputs to load
        WebDriverWait(self.driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input[date-format="M/D/YYYY"]'))
        )

        # Locate input elements
        date_inputs = self.driver.find_elements(By.CSS_SELECTOR, 'input[date-format="M/D/YYYY"]')
        time_inputs = self.driver.find_elements(By.CSS_SELECTOR, 'input[type="text"]')

        start_date_input = date_inputs[0]
        end_date_input = date_inputs[1]
        start_time_input = time_inputs[0]
        end_time_input = time_inputs[1]

        # Read current end datetime from page
        end_date_str = end_date_input.get_attribute("value")  # e.g. '6/10/2025'
        end_time_str = end_time_input.get_attribute("value")  # e.g. '14:55'
        end_dt = datetime.strptime(f"{end_date_str} {end_time_str}", "%m/%d/%Y %H:%M")

        # Compute start datetime (6 hours earlier, rounded to nearest 5 minutes)
        start_dt = end_dt - timedelta(hours=6)
        start_dt = start_dt.replace(minute=(start_dt.minute // 5) * 5, second=0, microsecond=0)

        # Format for input
        start_date_str = f"{start_dt.month}/{start_dt.day}/{start_dt.year}"
        start_time_str = start_dt.strftime("%H:%M")

        # Fill form using JS to trigger Angular model updates
        self._force_fill_time(start_time_input, start_time_str)
        self._force_fill_time(start_date_input, start_date_str)

        # Trigger Angular to acknowledge form changes
        end_date_input.click()  # defocus start_time_input

        print(f"Filled form from {start_date_str} {start_time_str} to {end_date_str} {end_time_str}")

        # Submit
        submit_button = self.driver.find_element(By.XPATH, '//button[contains(text(), "Submit")]')
        submit_button.click()

        time.sleep(2)

        # Now click Export
        export_button = WebDriverWait(self.driver, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.dm-download'))
        )
        before = set(os.listdir(self.download_dir))
        export_button.click()

        file_path = self._wait_for_file_download(before, timeout=self.download_wait)
        df = pd.read_csv(file_path)
        os.remove(file_path)
        return df

In [37]:
def clean_forecast_data(df, forecast_area='RTO_COMBINED'):
    """
    Filters and standardizes forecast load data for a given area.

    Parameters:
        df (pd.DataFrame): Raw forecast data.
        forecast_area (str): Forecast zone/region to filter. Default is 'RTO_COMBINED'.

    Returns:
        pd.DataFrame: Cleaned forecast data with standardized column names.
    """
    df = df[df['forecast_area'] == forecast_area]
    df = df[['evaluated_at_utc', 'forecast_datetime_beginning_utc', 'forecast_load_mw']]
    df = df.rename(columns={
        'forecast_datetime_beginning_utc': 'target_time',
        'evaluated_at_utc': 'issued_at',
        'forecast_load_mw': 'forecast_load'
    })

    df['issued_at'] = df['issued_at'].apply(parse_datetime)
    df['target_time'] = df['target_time'].apply(parse_datetime)

    issued_fail = df['issued_at'].isna().sum()
    target_fail = df['target_time'].isna().sum()
    if issued_fail or target_fail:
        print(f"Warning: {issued_fail} 'issued_at' and {target_fail} 'target_time' values failed to parse.")

    df = df.sort_values(by=['issued_at', 'target_time']).reset_index(drop=True)

    df["horizon"] = ((df['target_time'] - df['issued_at']) / pd.Timedelta(minutes=5)).astype(int)

    latest_issued_at = df['issued_at'].max()
    df = df[df['issued_at'] < latest_issued_at]



    # df['issued_at_str'] = df['issued_at'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
    # df['target_time_str'] = df['target_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
    return df

In [29]:
fetcher = PJMDataFetcher(download_dir=r'D:\ML\EnergyPrediction\deploy\storage', headless=True)
actual = fetcher.fetch_recent_actual()
past_buffer = fetcher.fetch_historical_actual()
forecast = fetcher.fetch_recent_forecast()

Filled form from 6/15/2025 02:40 to 6/15/2025 08:40


In [30]:
forecast.head()

,evaluated_at_utc,evaluated_at_ept,forecast_datetime_beginning_utc,forecast_datetime_beginning_ept,forecast_datetime_ending_utc,forecast_datetime_ending_ept,forecast_area,forecast_load_mw
0,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:45:00 PM,6/15/2025 8:45:00 AM,AE/MIDATL,997.0
1,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:45:00 PM,6/15/2025 8:45:00 AM,6/15/2025 12:50:00 PM,6/15/2025 8:50:00 AM,AE/MIDATL,996.0
2,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:50:00 PM,6/15/2025 8:50:00 AM,6/15/2025 12:55:00 PM,6/15/2025 8:55:00 AM,AE/MIDATL,996.0
3,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:55:00 PM,6/15/2025 8:55:00 AM,6/15/2025 1:00:00 PM,6/15/2025 9:00:00 AM,AE/MIDATL,995.0
4,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 1:00:00 PM,6/15/2025 9:00:00 AM,6/15/2025 1:05:00 PM,6/15/2025 9:05:00 AM,AE/MIDATL,995.0


In [31]:
forecast['forecast_area'].unique()

array(['AE/MIDATL', 'AEP', 'AP', 'BG&E/MIDATL', 'COMED', 'PPL/MIDATL',
       'PENELEC/MIDATL', 'PEPCO/MIDATL', 'PSE&G/MIDATL', 'RECO/MIDATL',
       'RTO_COMBINED', 'SOUTHERN_REGION', 'UGI/MIDATL', 'WESTERN_REGION',
       'DEOK', 'DOMINION', 'DP&L/MIDATL', 'DAYTON', 'JCP&L/MIDATL',
       'METED/MIDATL', 'PECO/MIDATL', 'MID_ATLANTIC_REGION', 'DUQUESNE',
       'EKPC', 'ATSI'], dtype=object)

In [32]:
forecast.columns

Index(['evaluated_at_utc', 'evaluated_at_ept',
       'forecast_datetime_beginning_utc', 'forecast_datetime_beginning_ept',
       'forecast_datetime_ending_utc', 'forecast_datetime_ending_ept',
       'forecast_area', 'forecast_load_mw'],
      dtype='object')

In [33]:
forecast_filtered = forecast[forecast['forecast_area'] == 'RTO_COMBINED']

In [36]:
forecast_filtered

,evaluated_at_utc,evaluated_at_ept,forecast_datetime_beginning_utc,forecast_datetime_beginning_ept,forecast_datetime_ending_utc,forecast_datetime_ending_ept,forecast_area,forecast_load_mw
210,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:45:00 PM,6/15/2025 8:45:00 AM,RTO_COMBINED,78472.0
211,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:45:00 PM,6/15/2025 8:45:00 AM,6/15/2025 12:50:00 PM,6/15/2025 8:50:00 AM,RTO_COMBINED,78755.0
212,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:50:00 PM,6/15/2025 8:50:00 AM,6/15/2025 12:55:00 PM,6/15/2025 8:55:00 AM,RTO_COMBINED,79038.0
213,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 12:55:00 PM,6/15/2025 8:55:00 AM,6/15/2025 1:00:00 PM,6/15/2025 9:00:00 AM,RTO_COMBINED,79321.0
214,6/15/2025 12:40:00 PM,6/15/2025 8:40:00 AM,6/15/2025 1:00:00 PM,6/15/2025 9:00:00 AM,6/15/2025 1:05:00 PM,6/15/2025 9:05:00 AM,RTO_COMBINED,79601.0
...,...,...,...,...,...,...,...,...
5340,6/15/2025 12:00:00 PM,6/15/2025 8:00:00 AM,6/15/2025 1:35:00 PM,6/15/2025 9:35:00 AM,6/15/2025 1:40:00 PM,6/15/2025 9:40:00 AM,RTO_COMBINED,80655.0
5341,6/15/2025 12:00:00 PM,6/15/2025 8:00:00 AM,6/15/2025 1:40:00 PM,6/15/2025 9:40:00 AM,6/15/2025 1:45:00 PM,6/15/2025 9:45:00 AM,RTO_COMBINED,80912.0
5342,6/15/2025 12:00:00 PM,6/15/2025 8:00:00 AM,6/15/2025 1:45:00 PM,6/15/2025 9:45:00 AM,6/15/2025 1:50:00 PM,6/15/2025 9:50:00 AM,RTO_COMBINED,81167.0
5343,6/15/2025 12:00:00 PM,6/15/2025 8:00:00 AM,6/15/2025 1:50:00 PM,6/15/2025 9:50:00 AM,6/15/2025 1:55:00 PM,6/15/2025 9:55:00 AM,RTO_COMBINED,81418.0


In [38]:
forecast_data = clean_forecast_data(forecast)

In [39]:
forecast_data

,issued_at,target_time,forecast_load,horizon
0,2025-06-15 12:00:00,2025-06-15 12:00:00,75280.0,0
1,2025-06-15 12:00:00,2025-06-15 12:05:00,75552.0,1
2,2025-06-15 12:00:00,2025-06-15 12:10:00,75826.0,2
3,2025-06-15 12:00:00,2025-06-15 12:15:00,76107.0,3
4,2025-06-15 12:00:00,2025-06-15 12:20:00,76393.0,4
...,...,...,...,...
187,2025-06-15 12:35:00,2025-06-15 14:10:00,83198.0,19
188,2025-06-15 12:35:00,2025-06-15 14:15:00,83427.0,20
189,2025-06-15 12:35:00,2025-06-15 14:20:00,83654.0,21
190,2025-06-15 12:35:00,2025-06-15 14:25:00,83884.0,22
